# MNIST-10 Quantum Counting Demo

This notebook trains a small classical model, loads its weights into the
QArgus quantum model, runs inference, and applies a quantum-counting
style estimator to count a target digit. It follows the Regime-3 semantic
pipeline from:

A. G. Rattew, P.-W. Huang, N. Guo, L. A. Pira, and P. Rebentrost,
"Accelerating Inference for Multilayer Neural Networks with Quantum Computers"
(arXiv:2510.07195, 2025).


In [ ]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.linear_model import SGDClassifier
import sys
from pathlib import Path

# Resolve repo root when running from either the repo or notebooks/ dir.
ROOT = Path.cwd()
if not (ROOT / "src").exists():
    ROOT = ROOT.parent
SRC = ROOT / "src"
if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))
from qargus import QuantumResNet, QuantumResNetConfig, estimate_count, run_semantic_model, model_to_qasm
from qargus.weights import load_sklearn_classifier


In [ ]:
mnist = fetch_openml("mnist_784", version=1, as_frame=False, parser="liac-arff")
X_full = mnist.data.astype(np.float32).reshape(-1, 28, 28) / 255.0
y = mnist.target.astype(int)

X_crop = X_full[:, 2:26, 2:26]
X_down = X_crop.reshape(-1, 8, 3, 8, 3).mean(axis=(2, 4))
X_img = X_down[:, None, :, :]
X = X_img.reshape(-1, 64)

zero_mask = np.isclose(np.linalg.norm(X, axis=1), 0.0)
if np.any(zero_mask):
    X[zero_mask, 0] = 1e-6
    X_img[zero_mask, 0, 0, 0] = 1e-6

rng = np.random.default_rng(0)
train_idx = rng.choice(len(X), size=1000, replace=False)
remaining = np.setdiff1d(np.arange(len(X)), train_idx, assume_unique=False)
test_idx = rng.choice(remaining, size=128, replace=False)

X_train = X[train_idx]
y_train = y[train_idx]
X_test = X[test_idx]
y_test = y[test_idx]
X_test_img = X_img[test_idx]

print("Train size:", X_train.shape[0])
print("Test size:", X_test.shape[0])


In [ ]:
clf = SGDClassifier(
    loss="log_loss",
    fit_intercept=False,
    max_iter=2000,
    tol=1e-3,
    random_state=0,
)
clf.fit(X_train, y_train)

train_acc = np.mean(clf.predict(X_train) == y_train)
test_acc = np.mean(clf.predict(X_test) == y_test)
print(f"Train accuracy: {train_acc:.3f}")
print(f"Test accuracy: {test_acc:.3f}")


In [ ]:
config = QuantumResNetConfig(
    input_shape=(1, 8, 8),
    num_classes=10,
    num_blocks=3,
    channels=1,
    kernel_size=3,
    output_mode="classifier",
)
model = QuantumResNet(config)
load_sklearn_classifier(model, clf)

preds = []
quantum_logits = []
for img in X_test_img:
    result = model.forward(img)
    quantum_logits.append(result.logits)
    preds.append(int(np.argmax(result.probabilities)))

target_digit = 3
marks = [p == target_digit for p in preds]
qc = estimate_count(marks, precision_bits=6)

print("Target digit:", target_digit)
print("True count:", qc.true_count)
print("Estimated count:", qc.estimated_count)
print("Oracle queries (simulated):", qc.oracle_queries)


## Deploy with semantic execution and QASM export

Semantic execution runs the same pipeline without circuit synthesis.
QASM export uses Qiskit/Aer to compile the unitary circuit for a concrete input.


In [ ]:
sample = X_test_img[0]
semantic = run_semantic_model(model, sample)
forward = model.forward(sample)

print('Semantic logits (L2-normalized):', semantic.state[:5])
print('Forward logits (L2-normalized):', forward.logits[:5])
print('Max diff:', np.max(np.abs(semantic.state - forward.logits)))


In [ ]:
qasm = model_to_qasm(model, input_state=sample, include_measurements=False)
qasm_path = ROOT / "notebooks" / "mnist10_semantic_unitary.qasm"
qasm_path.write_text(qasm)
print("Wrote QASM to:", qasm_path)
print(qasm.splitlines()[0])


In [ ]:
def _l2_normalize_rows(arr):
    norms = np.linalg.norm(arr, axis=1, keepdims=True)
    return arr / np.clip(norms, 1e-12, None)

X_test_norm = _l2_normalize_rows(X_test)
classical_logits_raw = X_test_norm @ clf.coef_.T
classical_logits = _l2_normalize_rows(classical_logits_raw)
classical_preds = np.argmax(classical_logits, axis=1)

agreement = np.mean(classical_preds == np.array(preds))
max_logit_diff = np.max(np.abs(classical_logits - np.vstack(quantum_logits)))
max_raw_diff = np.max(np.abs(classical_logits_raw - np.vstack(quantum_logits)))
print(f"Quantum/classical agreement: {agreement:.3f}")
print(f"Max logit diff (normalized): {max_logit_diff:.6f}")

for i in range(5):
    print(i, classical_preds[i], preds[i])


## Resource estimation and advantage signals

This section summarizes semantic execution metadata, circuit gate counts,
and a classical baseline for comparison.


In [ ]:
from qargus import build_regime3_unitary_circuit
from qiskit import transpile
from qiskit_aer import Aer

circuit = build_regime3_unitary_circuit(model)
backend = Aer.get_backend("aer_simulator")
transpiled = transpile(circuit, backend=backend, optimization_level=1)
counts = transpiled.count_ops()
two_qubit = sum(counts.get(g, 0) for g in ("cx", "cz", "swap"))
print("Circuit depth:", transpiled.depth())
print("Two-qubit gates:", two_qubit)
print("Gate counts:", counts)
